# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb --quiet
!pip install datasets transformers accelerate --quiet
!pip install intervaltree stanza textstat --quiet

In [ ]:
# Base imports
import math, datetime, os, shutil, itertools, wandb, json, random, re, csv
import numpy as np
import xml.etree.ElementTree as ET
# Transformer model imports
from transformers import AutoTokenizer, DataCollatorForWholeWordMask, TrainingArguments, Trainer, AutoModelForMaskedLM, EarlyStoppingCallback
import datasets, transformers, torch
# NER imports
import nltk
import intervaltree, stanza, textstat
from intervaltree import Interval, IntervalTree

In [ ]:
WANDB_API_KEY = '7425c67d3c5151a3744fe900a66cc0a3850c0858'
wandb.login(key=WANDB_API_KEY)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
MODEL_NAME = 'emilyalsentzer/Bio_ClinicalBERT'
MODEL_TITLE = MODEL_NAME.split('/')[-1]

REPO_PATH = 'drive/MyDrive/LiboMsc'
MODEL_PATH = f'{REPO_PATH}/{MODEL_TITLE}'

TRAIN_DATASET_PATH = f'{REPO_PATH}/data/i2b2_2024_T1_train'

CHUNK_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
TEST_MASK_RATIOS = {
  # PHI is set to 1.0 and MED_* to 0.0
  # Every other missing annotation type is automatically set to 0.0
  'NOUN': 0.70,
  'VERB': 0.70,
  'ADJ': 0.70,
}

In [ ]:
if not os.path.exists(MODEL_PATH):
    os.mkdir(MODEL_PATH)
    print('Folder created.')

# Dataset

### Load dataset

In [ ]:
def parse_xml_file(file_path):
  ''' Takes an XML file path and extracts note_id and text. '''
  tree = ET.parse(file_path)
  root = tree.getroot()
  data = {
    'note_id': file_path.split('/')[-1].replace('.xml', ''),
    'text':    root.find('TEXT').text,
  }
  return data


def load_xml_folder(folder_path):
  ''' Takes a folder path and loads all XML files from it. '''
  data_list = []
  for filename in os.listdir(folder_path):
    if filename.endswith('.xml'):
      file_path = os.path.join(folder_path, filename)
      data = parse_xml_file(file_path)
      data_list.append(data)

  return data_list

In [ ]:
# Load training dataset
raw_train_dataset = datasets.Dataset.from_list(load_xml_folder(TRAIN_DATASET_PATH))

# Split training dataset into training and validation
train_dataset = raw_train_dataset.train_test_split(test_size=0.2, seed=42)

### Extract annotations (PHIs, NER, and POS tags)

#### Prepare PHIs

In [ ]:
# Download Philter and dependencies
!git clone https://github.com/BCHSI/philter-deidstable1_mirror.git ./philter/src
%cd ./philter/src
!git checkout v1.2024.1
!pip install -r requirements.txt
%cd ../..

nltk.download('averaged_perceptron_tagger', quiet=True)

In [ ]:
# Format all letters from training dataset
%rm -rf ./philter/results ./philter/data
os.makedirs('./philter/results')
os.makedirs('./philter/data')

for letter in raw_train_dataset:
  with open(f"./philter/data/{letter['note_id']}.txt", 'w+') as f:
    f.write(letter['text'])

In [ ]:
# Extract PHIs from each letter
%cd philter/src
!python3 deidpipe.py -i ../data/ -o ../results/ -f configs/philter_one2024.json -d False
%cd ../..

Streaming output truncated to the last 5000 lines.
Successfully parsed: 7154 dates.
Failed to parse: 174 dates.
/content


In [ ]:
# Save PHIs in each sample of the dataset
def get_phi(sample):
  with open(f'./philter/results/log/phi_marked.json', 'r') as f:
    phi = json.load(f)
  phis = phi[f'../data/{sample["note_id"]}.txt']
  for p in phis:
    p['label'] = f'PHI_{p["type"]}'
    p.pop('type')
    p['text'] = p.pop('word')
    p.pop('context')
  return phis

#### Prepare NER and POS

In [ ]:
# Load model to annotate clinical text
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
stza_detector = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package        |
------------------------------------
| tokenize        | mimic          |
| pos             | mimic_charlm   |
| lemma           | mimic_nocharlm |
| depparse        | mimic_charlm   |
| ner             | i2b2           |
| backward_charlm | mimic          |
| pretrain        | mimic          |
| forward_charlm  | mimic          |



INFO:stanza:Downloaded file to /root/stanza_resources/en/tokenize/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/pos/mimic_charlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/lemma/mimic_nocharlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/depparse/mimic_charlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/ner/i2b2.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/backward_charlm/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/pretrain/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/forward_charlm/mimic.pt
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | mimic          |
| pos       | mimic_charlm   |
| lemma     | mimic_nocharlm |
| depparse  | mimic_charlm   |
| ner       | i2b2           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary 

In [ ]:
def get_all_annotations(text, phi):
  doc = stza_detector(text)
  annotations = phi.copy()

  # Get all medical entities (to keep) that do not overlap with PHIs (to remove)
  tree_phi = IntervalTree.from_tuples([(inf['start'], inf['end']) for inf in phi])
  def overlaps(tree, start_ind, end_ind):
    return len(tree.overlap(start_ind, end_ind)) > 0

  for ents in doc.entities:
    if not overlaps(tree_phi, ents.start_char, ents.end_char):
      annotations.append(dict(start=ents.start_char, end=ents.end_char, text=ents.text, label=f'MED_{ents.type}'))
  # print('Number of medical entities:', len(annotations))

  # Search for other types of entities (NOUN, ADJ, VRB...) that do not overlap with medical entities (to keep)
  tree_med = IntervalTree.from_tuples([(annotation['start'], annotation['end']) for annotation in annotations])

  for sent in doc.sentences:
    for word in sent.words:
      if (not overlaps(tree_med, word.start_char, word.end_char)) and (not overlaps(tree_phi, word.start_char, word.end_char)):
        annotations.append(dict(start=word.start_char, end=word.end_char, text=word.text, label=word.upos))
  # print('Number of all entities (medical + others):', len(annotations))

  # Sort annotations based on appeareance order
  annotations = sorted(annotations, key=lambda x: x['start'])

  # Decompose multi-word annotations into multiple single-word annotations
  sing_word_annotations = []
  for anno in annotations:
    # If annotation is a NUM or PUNCT keep it as it is
    if anno['label'] == 'NUM' or anno['label'] == 'PUNCT':
      sing_word_annotations.append(anno)
    # Otherwise decompose it
    else:
      indexes = [[]]
      for i, char in enumerate(anno['text']):
        if char.isalpha() or char.isalnum() or char == '\'':
          indexes[-1].append(i)
        else:
          indexes.append([])

      for new_anno in [{
          'start': anno['start']+i[0], 'end': anno['start']+i[-1]+1,
          'text': ''.join([anno['text'][e] for e in i]), 'label': anno['label']
        } for i in indexes if len(i)>0]:
          sing_word_annotations.append(new_anno)

  # Sort annotations based on appeareance order
  sing_word_annotations = sorted(sing_word_annotations, key=lambda x: x['start'])

  return sing_word_annotations

#### Extraction

In [ ]:
def extract_all_annotations(sample):
    phis = get_phi(sample)
    all_annotations = get_all_annotations(sample['text'], phis)
    sample['annotations'] = all_annotations
    return sample


def extract_annotations_to_mask_only(sample, mask_ratios):
    # Get all annotations
    phis = get_phi(sample)
    all_annotations = get_all_annotations(sample['text'], phis)

    # Sort annotations
    annotations = {'PHI': [], 'MED': []}
    for anno in all_annotations:
        anno_type = anno['label']
        if anno_type.startswith('PHI'):
            annotations['PHI'].append(anno)
        elif anno_type.startswith('MED'):
            annotations['MED'].append(anno)
        else:
            if anno_type not in annotations:
                annotations[anno_type] = []
            annotations[anno_type].append(anno)

    # Select entities to mask based on given ratios
    entities_to_mask = {}
    for anno_type, annos in annotations.items():
        if anno_type == 'PHI':
            # Mask ratio for PHI is 1.0
            entities_to_mask[anno_type] = annos.copy()
        elif (anno_type == 'MED') or (anno_type not in mask_ratios):
            # Mask ratio for MED or absent types are 0.0
            continue
        else:
            random.seed(55) # For reproducibility
            annotations_to_keep = random.sample(annos, int(len(annos) * mask_ratios[anno_type]))
            entities_to_mask[anno_type] = annotations_to_keep

    # Merge all entities (stop considering annotation types) and order them
    entities_to_mask = sorted(sum(entities_to_mask.values(), []), key=lambda x: x['start'])

    sample['annotations'] = entities_to_mask
    return sample

In [ ]:
# Extract annotations
train_dataset['train'] = train_dataset['train'].map(extract_all_annotations)
train_dataset['test'] = train_dataset['test'].map(extract_annotations_to_mask_only, fn_kwargs={'mask_ratios': TEST_MASK_RATIOS})

train_dataset

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['note_id', 'text', 'annotations'],
        num_rows: 632
    })
    test: Dataset({
        features: ['note_id', 'text', 'annotations'],
        num_rows: 158
    })
})

### Tokenize dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def align_annotations_with_tokens(tokenized_text, annotations):
  tokens = tokenized_text[0].tokens
  aligned_annotations = ["O"]*len(tokens)

  for anno in (annotations):
      for char_ix in range(anno['start'],anno['end']):
          token_ix = tokenized_text.char_to_token(char_ix)
          if token_ix is not None: # White spaces have no token and will return None
              aligned_annotations[token_ix] = anno['label']

  # for token, anno in zip(tokens, aligned_annotations):
  #   print(token, '-', anno)

  return aligned_annotations

In [ ]:
def tokenize_function(sample):
  # Get annotations and tokenized text
  annotations = sample['annotations']
  sample = tokenizer(sample['text'])

  # Align annotations with tokens
  aligned_annotations = align_annotations_with_tokens(tokenized_text=sample, annotations=annotations)
  sample['annotations'] = aligned_annotations

  return sample

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=False, remove_columns=train_dataset['train'].column_names)
tokenized_train_dataset

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['annotations', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 632
    })
    test: Dataset({
        features: ['annotations', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 158
    })
})

### Group dataset

In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than CHUNK_SIZE
    total_length = (total_length // CHUNK_SIZE) * CHUNK_SIZE
    # Split by chunks of max_len
    result = {
        k: [t[i : i + CHUNK_SIZE] for i in range(0, total_length, CHUNK_SIZE)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
grp_datasets = tokenized_train_dataset.map(group_texts, batched=True)
grp_datasets

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['annotations', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2896
    })
    test: Dataset({
        features: ['annotations', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 720
    })
})

# Hyperparameter tuning & Training

## Finetuning setup

### Data Collator

In [ ]:
from transformers.data.data_collator import *
from transformers.data.data_collator import _torch_collate_batch, _tf_collate_batch, _numpy_collate_batch

@dataclass
class CustomBaseWholeWordMaskingDataCollator(DataCollatorForWholeWordMask):
    def __init__(self, tokenizer, mlm_probability):
        super().__init__(tokenizer, mlm_probability)

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
      if isinstance(examples[0], Mapping):
          input_ids = [e["input_ids"] for e in examples]
      else:
          input_ids = examples
          examples = [{"input_ids": e} for e in examples]

      batch_input = _torch_collate_batch(input_ids, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)

      mask_labels = []
      for e in examples:
          ref_tokens = []
          for id in tolist(e["input_ids"]):
              token = self.tokenizer._convert_id_to_token(id)
              ref_tokens.append(token)

          # For Chinese tokens, we need extra inf to mark sub-word, e.g [喜,欢]-> [喜，##欢]
          if "chinese_ref" in e:
              ref_pos = tolist(e["chinese_ref"])
              len_seq = len(e["input_ids"])
              for i in range(len_seq):
                  if i in ref_pos:
                      ref_tokens[i] = "##" + ref_tokens[i]
          mask_labels.append(self._whole_word_mask(ref_tokens, e['annotations']))
      batch_mask = _torch_collate_batch(mask_labels, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
      inputs, labels = self.torch_mask_tokens(batch_input, batch_mask)
      return {"input_ids": inputs, "labels": labels}

    def tf_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
      if isinstance(examples[0], Mapping):
          input_ids = [e["input_ids"] for e in examples]
      else:
          input_ids = examples
          examples = [{"input_ids": e} for e in examples]

      batch_input = _tf_collate_batch(input_ids, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)

      mask_labels = []
      for e in examples:
          ref_tokens = []
          for id in tolist(e["input_ids"]):
              token = self.tokenizer._convert_id_to_token(id)
              ref_tokens.append(token)

          # For Chinese tokens, we need extra inf to mark sub-word, e.g [喜,欢]-> [喜，##欢]
          if "chinese_ref" in e:
              ref_pos = tolist(e["chinese_ref"])
              len_seq = len(e["input_ids"])
              for i in range(len_seq):
                  if i in ref_pos:
                      ref_tokens[i] = "##" + ref_tokens[i]
          mask_labels.append(self._whole_word_mask(ref_tokens, e['annotations']))
      batch_mask = _tf_collate_batch(mask_labels, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
      inputs, labels = self.tf_mask_tokens(tf.cast(batch_input, tf.int64), batch_mask)
      return {"input_ids": inputs, "labels": labels}

    def numpy_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
      if isinstance(examples[0], Mapping):
          input_ids = [e["input_ids"] for e in examples]
      else:
          input_ids = examples
          examples = [{"input_ids": e} for e in examples]

      batch_input = _numpy_collate_batch(input_ids, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)

      mask_labels = []
      for e in examples:
          ref_tokens = []
          for id in tolist(e["input_ids"]):
              token = self.tokenizer._convert_id_to_token(id)
              ref_tokens.append(token)

          # For Chinese tokens, we need extra inf to mark sub-word, e.g [喜,欢]-> [喜，##欢]
          if "chinese_ref" in e:
              ref_pos = tolist(e["chinese_ref"])
              len_seq = len(e["input_ids"])
              for i in range(len_seq):
                  if i in ref_pos:
                      ref_tokens[i] = "##" + ref_tokens[i]
          mask_labels.append(self._whole_word_mask(ref_tokens, e['annotations']))
      batch_mask = _numpy_collate_batch(mask_labels, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
      inputs, labels = self.numpy_mask_tokens(batch_input, batch_mask)
      return {"input_ids": inputs, "labels": labels}



@dataclass
class CustomTrainingWordMaskingDataCollator(CustomBaseWholeWordMaskingDataCollator):
    def __init__(self, tokenizer, mlm_probability, phi_masking_proportion):
        self.phi_masking_proportion = phi_masking_proportion
        super().__init__(tokenizer, mlm_probability)

    def _whole_word_mask(self, input_tokens: List[str], token_labels: List[str], max_predictions=512):
        """
        Get 0/1 labels for masked tokens with whole word mask proxy
        """
        if not isinstance(self.tokenizer, (BertTokenizer, BertTokenizerFast)):
            warnings.warn(
                "DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. "
                "Please refer to the documentation for more information."
            )

        phi_indexes, cand_indexes = [], []
        for i, (token, label) in enumerate(zip(input_tokens, token_labels)):
            if token == "[CLS]" or token == "[SEP]":
                continue

            # Force mask PHIs
            if label.startswith("PHI"):
                if len(phi_indexes) >= 1 and token.startswith("##"):
                    phi_indexes[-1].append(i)
                else:
                    phi_indexes.append([i])
            # Otherwise, do not mask if medical entity or is not a noun
            elif label.startswith("MED") or label == 'PUNCT': # TODO: or label != "NOUN" ????
              continue
            # Finally, if non-medical noun: mask
            else:
                if len(cand_indexes) >= 1 and token.startswith("##"):
                    cand_indexes[-1].append(i)
                else:
                    cand_indexes.append([i])

        # Select a proportion of PHIs to mask
        random.shuffle(phi_indexes)
        phi_indexes = phi_indexes[:int(len(phi_indexes)*self.phi_masking_proportion)]

        # Build candidate indexes and /!\ Prioritize PHIs to mask /!\
        random.shuffle(cand_indexes)
        cand_indexes = phi_indexes + cand_indexes

        num_to_predict = min(max_predictions, max(1, int(round(len(input_tokens) * self.mlm_probability))))
        masked_lms = []
        covered_indexes = set()
        for index_set in cand_indexes:
            if len(masked_lms) >= num_to_predict:
                break
            # If adding a whole-word mask would exceed the maximum number of
            # predictions, then just skip this candidate.
            if len(masked_lms) + len(index_set) > num_to_predict:
                continue
            is_any_index_covered = False
            for index in index_set:
                if index in covered_indexes:
                    is_any_index_covered = True
                    break
            if is_any_index_covered:
                continue
            for index in index_set:
                covered_indexes.add(index)
                masked_lms.append(index)

        if len(covered_indexes) != len(masked_lms):
            raise ValueError("Length of covered_indexes is not equal to length of masked_lms.")
        mask_labels = [1 if i in covered_indexes else 0 for i in range(len(input_tokens))]
        return mask_labels



@dataclass
class CustomEvaluationWordMaskingDataCollator(CustomBaseWholeWordMaskingDataCollator):
    def __init__(self, tokenizer, mlm_probability=1.0):
        super().__init__(tokenizer, mlm_probability)

    def _whole_word_mask(self, input_tokens: List[str], token_labels: List[str], max_predictions=512):
        """
        Get 0/1 labels for masked tokens with whole word mask proxy
        """
        if not isinstance(self.tokenizer, (BertTokenizer, BertTokenizerFast)):
            warnings.warn(
                "DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. "
                "Please refer to the documentation for more information."
            )

        cand_indexes = []
        for i, (token, label) in enumerate(zip(input_tokens, token_labels)):
            if token == "[CLS]" or token == "[SEP]":
                continue

            # Force mask all non-O annotations
            if label != 'O':
                if len(cand_indexes) >= 1 and token.startswith("##"):
                    cand_indexes[-1].append(i)
                else:
                    cand_indexes.append([i])

        random.shuffle(cand_indexes)

        num_to_predict = min(max_predictions, max(1, int(round(len(input_tokens) * self.mlm_probability))))
        masked_lms = []
        covered_indexes = set()
        for index_set in cand_indexes:
            if len(masked_lms) >= num_to_predict:
                break
            # If adding a whole-word mask would exceed the maximum number of
            # predictions, then just skip this candidate.
            if len(masked_lms) + len(index_set) > num_to_predict:
                continue
            is_any_index_covered = False
            for index in index_set:
                if index in covered_indexes:
                    is_any_index_covered = True
                    break
            if is_any_index_covered:
                continue
            for index in index_set:
                covered_indexes.add(index)
                masked_lms.append(index)

        if len(covered_indexes) != len(masked_lms):
            raise ValueError("Length of covered_indexes is not equal to length of masked_lms.")
        mask_labels = [1 if i in covered_indexes else 0 for i in range(len(input_tokens))]
        return mask_labels

In [ ]:
from transformers.trainer import *

class DoubleCollatorTrainer(Trainer):
    def __init__(
        self,
        model: Union[PreTrainedModel, nn.Module] = None,
        args: TrainingArguments = None,
        data_collator: Optional[DataCollator] = None,
        eval_data_collator: Optional[DataCollator] = None,
        train_dataset: Optional[Union[Dataset, IterableDataset, "datasets.Dataset"]] = None,
        eval_dataset: Optional[Union[Dataset, Dict[str, Dataset], "datasets.Dataset"]] = None,
        tokenizer: Optional[PreTrainedTokenizerBase] = None,
        model_init: Optional[Callable[[], PreTrainedModel]] = None,
        compute_metrics: Optional[Callable[[EvalPrediction], Dict]] = None,
        callbacks: Optional[List[TrainerCallback]] = None,
        optimizers: Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR] = (None, None),
        preprocess_logits_for_metrics: Optional[Callable[[torch.Tensor, torch.Tensor], torch.Tensor]] = None,
    ):
        super().__init__(
            model, args, data_collator, train_dataset, eval_dataset, tokenizer,
            model_init, compute_metrics, callbacks, optimizers, preprocess_logits_for_metrics,
        )
        self.eval_data_collator = eval_data_collator


    def get_eval_dataloader(self, eval_dataset: Optional[Union[str, Dataset]] = None) -> DataLoader:
        """
        Returns the evaluation [`~torch.utils.data.DataLoader`].

        Subclass and override this method if you want to inject some custom behavior.

        Args:
            eval_dataset (`str` or `torch.utils.data.Dataset`, *optional*):
                If a `str`, will use `self.eval_dataset[eval_dataset]` as the evaluation dataset. If a `Dataset`, will override `self.eval_dataset` and must implement `__len__`. If it is a [`~datasets.Dataset`], columns not accepted by the `model.forward()` method are automatically removed.
        """
        if eval_dataset is None and self.eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")

        # If we have persistent workers, don't do a fork bomb especially as eval datasets
        # don't change during training
        dataloader_key = eval_dataset if isinstance(eval_dataset, str) else "eval"
        if (
            hasattr(self, "_eval_dataloaders")
            and dataloader_key in self._eval_dataloaders
            and self.args.dataloader_persistent_workers
        ):
            return self.accelerator.prepare(self._eval_dataloaders[dataloader_key])

        eval_dataset = (
            self.eval_dataset[eval_dataset]
            if isinstance(eval_dataset, str)
            else eval_dataset
            if eval_dataset is not None
            else self.eval_dataset
        )
        data_collator = self.eval_data_collator

        if is_datasets_available() and isinstance(eval_dataset, datasets.Dataset):
            eval_dataset = self._remove_unused_columns(eval_dataset, description="evaluation")
        else:
            data_collator = self._get_collator_with_removed_columns(data_collator, description="evaluation")

        dataloader_params = {
            "batch_size": self.args.eval_batch_size,
            "collate_fn": data_collator,
            "num_workers": self.args.dataloader_num_workers,
            "pin_memory": self.args.dataloader_pin_memory,
            "persistent_workers": self.args.dataloader_persistent_workers,
        }

        if not isinstance(eval_dataset, torch.utils.data.IterableDataset):
            dataloader_params["sampler"] = self._get_eval_sampler(eval_dataset)
            dataloader_params["drop_last"] = self.args.dataloader_drop_last
            dataloader_params["prefetch_factor"] = self.args.dataloader_prefetch_factor

        # accelerator.free_memory() will destroy the references, so
        # we need to store the non-prepared version
        eval_dataloader = DataLoader(eval_dataset, **dataloader_params)
        if self.args.dataloader_persistent_workers:
            if hasattr(self, "_eval_dataloaders"):
                self._eval_dataloaders[dataloader_key] = eval_dataloader
            else:
                self._eval_dataloaders = {dataloader_key: eval_dataloader}

        return self.accelerator.prepare(eval_dataloader)

### Setup

In [ ]:
# Create a model initialiser function
def model_init():
    model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)
    # Make sure all tensors are contiguous
    for param in model.parameters():
        param.data = param.data.contiguous()
    return model

In [ ]:
def get_training_results_and_best_epoch(trainer):
  training_results = {'eval_loss': [], 'epoch': [], 'train_loss': [], 'eval_perplexity': []}
  for val, tr in zip(trainer.state.log_history[1::2], trainer.state.log_history[:-2:2]):
    training_results['eval_loss'].append(round(val['eval_loss'], 3))
    training_results['epoch'].append(val['epoch'])
    training_results['train_loss'].append(round(tr['loss'], 3))
    training_results['eval_perplexity'].append(round(math.exp(val['eval_loss']), 3))
  best_epoch = {key: val[np.argmin(training_results['eval_loss'])] for key, val in training_results.items()}
  return training_results, best_epoch

In [ ]:
def fine_tune(hyperparameters, dataset):
  ''' Function to fine-tune a pretrained model using Hugging Face's pipeline.
  '''
  # Model's and logs directory
  RUN_NAME = f"Run - {datetime.datetime.now().strftime('%m-%d-%H-%M')}"
  DIR = f"temp/hp_tuning/Results/{RUN_NAME}" # Models do not need to be saved during hyperparameter tuning
  os.environ["WANDB_PROJECT"] = 'MLM_LETTER' # set the wandb project where this run will be logged

  # Define training args
  training_args = TrainingArguments(
    run_name=RUN_NAME.replace(' ', '_'),
    output_dir=f"{DIR}/checkpoints",

    # Parameters
    per_device_train_batch_size = hyperparameters["batch_size"],
    learning_rate = hyperparameters["learning_rate"],
    weight_decay = hyperparameters['weight_decay'],
    num_train_epochs = 10,       # Use early stopping (so this is maximum epochs)
    fp16 = (DEVICE == 'cuda'),   # Use 16-bit (mixed) precision instead of 32-bit (ONLY POSSIBLE ON CUDA!)
    optim = "adamw_torch",
    seed=42,                     # Use a seed for reproducibility
    remove_unused_columns=False, # IMPORTANT: if not set to False, the custom data collator will delete 'annotations' column

    # Logging
    logging_dir=f"{DIR}/training_logs",
    logging_strategy="epoch",
    report_to="wandb",
    # Saving
    save_strategy="epoch",
    save_safetensors=True, save_total_limit=1,
    # Evaluating (Use validation loss for model selection and early stopping)
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss", greater_is_better=False,
  )

  # Create a Trainer instance
  trainer = DoubleCollatorTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=CustomTrainingWordMaskingDataCollator(
      tokenizer=tokenizer,
      mlm_probability=hyperparameters['train_masking_prob'],
      phi_masking_proportion=hyperparameters['phi_masking_proportion']
    ),
    eval_data_collator=CustomEvaluationWordMaskingDataCollator(tokenizer=tokenizer),
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(2, 0.0)],
  )

  # Fine-tune the model
  trainer.train()

  # Delete model checkpoints
  shutil.rmtree(DIR)

  # Get training results
  training_results, best_epoch = get_training_results_and_best_epoch(trainer)
  return training_results, best_epoch, RUN_NAME


## Hyperparameter tuning

In [ ]:
# Hyperparameters recommended by BERT and RoBERTa
HYPERPARAMETERS = {
    'weight_decay': [0.01], # 0.02
    'learning_rate': [1e-4, 5e-5, 3e-5],
    'batch_size': [8, 16], # 4
    'phi_masking_proportion': [0.75, 1.0],
    'train_masking_prob': [0.30, 0.50],
}

# Get all possible combination of hyperparameter sets (for grid search)
keys, values = zip(*HYPERPARAMETERS.items())
HYPERPARAMETERS_COMB = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(f'Number of hyperparameter sets to search through: {len(HYPERPARAMETERS_COMB)}.')

Number of hyperparameter sets to search through: 24.


In [ ]:
results = {}
with open(f'{MODEL_PATH}/training_logs.txt', 'a+') as log:

    # Fine-tune on each hyperparameter set (grid search)
    for HYPERPARAM in HYPERPARAMETERS_COMB:
        print('--- STARTING FINE-TUNING ---')
        training_results, best_epoch, run_name = fine_tune(hyperparameters=HYPERPARAM, dataset=grp_datasets)
        HYPERPARAM['epoch'] = best_epoch['epoch']
        results[best_epoch['eval_perplexity']] = HYPERPARAM

        print('Run name:', run_name)
        print('Hyperparameters:', HYPERPARAM)
        print('Final training results:', training_results)
        print('Best epoch results:', best_epoch)

        log.write(f"Run name: {run_name}\n")
        log.write('Hyperparameters:\n' + ',\n'.join([f'\t{key}={value}' for key, value in HYPERPARAM.items()]) + '.\n')
        log.write('Final training results:\n' + ';\n'.join([f'\t{key}={", ".join([str(i) for i in value])}' for key, value in training_results.items()]) + '.\n')
        log.write('Best epoch results:\n' + ',\n'.join([f'\t{key}={value}' for key, value in best_epoch.items()]) + '.\n\n')

--- STARTING FINE-TUNING ---


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

wandb: Currently logged in as: belkadisamuel. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.072400,3.100508
2,2.567300,2.909697
3,2.364400,2.805727
4,2.213700,2.762541
5,2.076400,2.699641
6,1.954700,2.666494
7,1.872000,2.690596
8,1.788300,2.589769
9,1.722900,2.654836
10,1.671500,2.604804


...

Epoch,Training Loss,Validation Loss
1,3.278800,3.300688
2,2.845400,3.114532
3,2.671300,3.000424
4,2.573900,2.934534
5,2.476200,2.852698
6,2.384000,2.844365
7,2.350900,2.827296
8,2.318800,2.778606
9,2.290000,2.816779
10,2.269700,2.752297


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Run name: Run - 09-08-17-45
Hyperparameters: {'weight_decay': 0.01, 'learning_rate': 3e-05, 'batch_size': 16, 'phi_masking_proportion': 0.75, 'train_masking_prob': 0.5, 'epoch': 10.0}
Final training results: {'eval_loss': [3.301, 3.115, 3.0, 2.935, 2.853, 2.844, 2.827, 2.779, 2.817, 2.752], 'epoch': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0], 'train_loss': [3.279, 2.845, 2.671, 2.574, 2.476, 2.384, 2.351, 2.319, 2.29, 2.27], 'eval_perplexity': [27.131, 22.523, 20.094, 18.813, 17.334, 17.191, 16.9, 16.097, 16.723, 15.679]}
Best epoch results: {'eval_loss': 2.752, 'epoch': 10.0, 'train_loss': 2.27, 'eval_perplexity': 15.679}
--- STARTING FINE-TUNING ---


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,3.543900,3.334927
2,3.076000,3.159799
3,2.875400,3.043682
4,2.767600,2.967408
5,2.661500,2.916734
6,2.562700,2.897597
7,2.515800,2.887901
8,2.487400,2.841732


## Training (with optimal hyperparameters)

In [ ]:
# Select best hyperparameter set
best_hyperparameters = results[min(results.keys())]
print(f'Best hyperparameters (prplx={min(results.keys())}):', best_hyperparameters)

In [ ]:
# Transform validation dataset into training format
train_dataset['test'] = train_dataset['test'].map(extract_all_annotations)
tokenized_train_dataset['test'] = train_dataset['test'].map(tokenize_function, batched=False, remove_columns=train_dataset['train'].column_names)
grp_datasets['test'] = tokenized_train_dataset['test'].map(group_texts, batched=True)

# Combine training and validation sets
grp_datasets_train_valid = datasets.concatenate_datasets([grp_datasets['train'], grp_datasets['test']])

In [ ]:
# Model's and logs directory
RUN_NAME = f"Run - {datetime.datetime.now().strftime('%m-%d-%H-%M')}"
DIR = f"{MODEL_PATH}/Results/{RUN_NAME}"
os.environ["WANDB_PROJECT"] = 'MLM_LETTER' # set the wandb project where this run will be logged

# Define training args
training_args = TrainingArguments(
  run_name=RUN_NAME.replace(' ', '_'),
  output_dir=f"{DIR}/checkpoints",

  # Parameters
  per_device_train_batch_size = best_hyperparameters["batch_size"],
  learning_rate = best_hyperparameters["learning_rate"],
  weight_decay = best_hyperparameters['weight_decay'],
  num_train_epochs = best_hyperparameters['epoch'],
  fp16 = (DEVICE == 'cuda'),   # Use 16-bit (mixed) precision instead of 32-bit (ONLY POSSIBLE ON CUDA!)
  optim = "adamw_torch",
  seed=56,                     # Use a seed for reproducibility
  remove_unused_columns=False, # IMPORTANT: if not set to False, the custom data collator will delete 'annotations' column

  # Logging, Saving, Evaluating
  logging_strategy="no",
  save_strategy="epoch", save_total_limit=1, save_safetensors=True,
  evaluation_strategy="no",
)

# Create a Trainer instance
trainer = DoubleCollatorTrainer(
  model_init=model_init,
  args=training_args,
  train_dataset=grp_datasets_train_valid,
  data_collator=CustomTrainingWordMaskingDataCollator(
    tokenizer=tokenizer,
    mlm_probability=best_hyperparameters['train_masking_prob'],
    phi_masking_proportion=best_hyperparameters['phi_masking_proportion']
  ),
  tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Step,Training Loss


In [ ]:
# Save logs for trained model hyperparameters
with open(f'{DIR}/best_model_hyperparameters.txt', 'w+') as f:
  f.write('Hyperparameters:\n' + ',\n'.join([f'\t{key}={value}' for key, value in best_hyperparameters.items()]) + '.\n\n')

In [ ]:
# Save tokenizer
tokenizer.save_pretrained(f'{DIR}/tokenizer')

# Close runtime (save compute units)

In [ ]:
# Close google colab runtime to save credits
from google.colab import runtime
runtime.unassign()